# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [13]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re





def nhl_correlation():
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    

    #Selecting year 2018 data
    nhl_df_2018 = nhl_df[nhl_df['year'] == 2018]
    #Selecting team, W and L
    nhl_df_2018 = nhl_df_2018[['team','W','L']]

    #Dropping division rows
    nhl_df_2018 = nhl_df_2018.drop([0,9,18,26], axis=0)

    #Converting columns to integer
    nhl_df_2018['W'] = [int(x) for x in nhl_df_2018['W']]
    nhl_df_2018['L'] = [int(x) for x in nhl_df_2018['L']]
    #Calculating Win/Loss Ratio
    nhl_df_2018['Ratio'] = (nhl_df_2018['W'])/(nhl_df_2018['W'] + nhl_df_2018['L'])
    len(nhl_df_2018['Ratio'])  
    nhl_df_2018['team'].value_counts()

    cities_df = cities[['Metropolitan area','Population (2016 est.)[8]','NHL']]
    cities_df = cities_df.rename(columns={'Population (2016 est.)[8]':'Population'})
    cities_df['Population'] = [int(x) for x in cities_df['Population']]


    #Function To remove special Characters
    def remove_char(x):
        x = str(x).strip()
        if '[note' in x:
            lst = x.split('[')
            if lst[0]=='':
                return np.nan
            return lst[0].strip()
        if '—' in x:
            x = np.nan
        return x

    #Function to find teams
    team_list = ['Red Wings','Maple Leafs','Blue Jackets','Golden Knights']
    def find_team(x):
        if x in team_list:
            return [x]
        pattern = '[A-Z][^A-Z]+'
        lst = re.findall(pattern, x)
        return lst

    cities_df['NHL'] = cities_df['NHL'].apply(remove_char)
    cities_df = cities_df.dropna()
    cities_df['NHL'] = cities_df['NHL'].apply(find_team)
    cities_df['NHL']

    def find_team_city(x):
        if 'Vegas' in x:
            return 'Vegas'
        if 'City' in x:
            return x.split('City')[0].strip()
        elif 'San Francisco' in x:
            return 'San Jose'
        elif '–' in x:
            return x.split('–')[0].strip()
        elif 'Bay' in x:
            return x.split('Bay')[0].strip()
        elif ',' in x:
            return x.split(',')[0].strip()

        return x


    cities_df['Team City'] = cities_df['Metropolitan area'].apply(find_team_city)
    cities_df


    nhl_df_2018['Team City'] = ''
    for l1,c1 in cities_df['Team City'].items():
        for l2,c2 in nhl_df_2018['team'].items():
            if c1 in (c2):
                nhl_df_2018['Team City'][l2] = c1
            elif c2 == 'Florida Panthers':
                nhl_df_2018['Team City'][l2] = 'Miami'
            elif c2 in 'New Jersey Devils*':
                nhl_df_2018['Team City'][l2] = 'New York'
            elif c2 == 'Carolina Hurricanes':
                nhl_df_2018['Team City'][l2] = 'Raleigh'
            elif c2 == 'Minnesota Wild*':
                nhl_df_2018['Team City'][l2] = 'Minneapolis'
            elif c2 == 'Colorado Avalanche*':
                nhl_df_2018['Team City'][l2] = 'Denver'
            elif c2 == 'Anaheim Ducks*':
                nhl_df_2018['Team City'][l2] = 'Los Angeles'
            elif c2 == 'Arizona Coyotes':
                nhl_df_2018['Team City'][l2] = 'Phoenix'




    df = pd.merge(cities_df, nhl_df_2018, how='inner', on='Team City')

    # df[['Metropolitan area', 'Population', 'Ratio']]
    result = df.groupby(['Metropolitan area','Population']).agg({'Ratio':np.mean})
    #result = result.set_index('Metropolitan area',axis=0)
    result = result.reset_index()
        # YOUR CODE HERE


    population_by_region = result['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = result['Ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"

    return result#stats.pearsonr(population_by_region, win_loss_by_region)[0]
    raise NotImplementedError()
nhl_correlation()

,Metropolitan area,Population,Ratio
0,Boston,4794447,0.714286
1,Buffalo,1132804,0.357143
2,Calgary,1392609,0.513889
3,Chicago,9512999,0.458333
4,Columbus,2041520,0.600000
5,Dallas–Fort Worth,7233323,0.567568
6,Denver,2853077,0.589041
7,Detroit,4297617,0.434783
8,Edmonton,1321426,0.473684
9,Las Vegas,2155664,0.680000


## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [17]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re





def nba_correlation():
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    #NBA 2018
    nba_df_2018 = nba_df[nba_df['year'] == 2018]
   

    #NBA Cities

    nba_city_df = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NBA']]
    nba_city_df =  nba_city_df.rename(columns={'Population (2016 est.)[8]':'Population'})
    nba_city_df['Population'] = [int(x) for x in nba_city_df['Population']]
    #

    def remove_char(x):
        x = str(x).strip()
        if '[note' in x:
            lst = x.split('[')
            if lst[0]=='':
                return np.nan
            return lst[0].strip()
        if '—' in x:
            x = np.nan
        return x

    nba_city_df['NBA'] = nba_city_df['NBA'].apply(remove_char)
    nba_city_df = nba_city_df.dropna()
    nba_city_df

    def nba_city(x):
        x = x.strip()
        if 'City' in x:
            return x.split('City')[0].strip()
        elif '–' in x:
            return x.split('–')[0].strip()
        elif 'Bay' in x:
            return x.split('Bay')[0].strip()
        elif ',' in x:
            return x.split(',')[0].strip()
        return x


    nba_city_df['Team City'] = nba_city_df['Metropolitan area'].apply(nba_city)
    nba_city_df
    nba_df_2018['Team City'] = ''
    for l1,c1 in nba_city_df['Team City'].items():
        for l2,c2 in nba_df_2018['team'].items():
            if c1 in c2:
                nba_df_2018['Team City'][l2] = c1
            elif 'Brooklyn Nets' in c2:
                nba_df_2018['Team City'][l2] = 'New York'
            elif 'Golden State Warriors' in c2:
                nba_df_2018['Team City'][l2] = 'San Francisco'
            elif 'Timberwolves' in c2:
                nba_df_2018['Team City'][l2] = 'Minneapolis'
            elif 'Jazz' in c2:
                nba_df_2018['Team City'][l2] = 'Salt Lake'
            elif 'Indiana' in c2:
                nba_df_2018['Team City'][l2] = 'Indianapolis'
            
            


    #Converting columns to integer
    nba_df_2018['W'] = [int(x) for x in nba_df_2018['W']]
    nba_df_2018['L'] = [int(x) for x in nba_df_2018['L']]

    nbadf = pd.merge(nba_city_df, nba_df_2018, how='inner', on='Team City')

    nbadf['Ratio'] = (nbadf['W'])/((nbadf['W'])+(nbadf['L']))

    nba_result_df = nbadf.groupby(['Metropolitan area', 'Population']).agg({'Ratio':np.mean})
    nba_result_df = nba_result_df.reset_index()
    nba_result_df

    #YOUR CODE HERE
    
    
    population_by_region = nba_result_df['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = nba_result_df['Ratio'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]
  
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    raise NotImplementedError()
    
#nba_correlation()

-0.17657160252844617

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [18]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re




           
        
def mlb_correlation():
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # YOUR CODE HERE
    mlb_df_2018 = mlb_df[mlb_df['year'] == 2018]
#mlb_df_2018

    mlb_city_df = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'MLB']]
    mlb_city_df =  mlb_city_df.rename(columns={'Population (2016 est.)[8]':'Population'})
    mlb_city_df['Population'] = [int(x) for x in mlb_city_df['Population']]

    def remove_char(x):
        x = str(x).strip()
        if '[note' in x:
            lst = x.split('[')
            if lst[0]=='':
                return np.nan
            return lst[0].strip()
        if '—' in x:
            x = np.nan
        return x

    mlb_city_df['MLB'] = mlb_city_df['MLB'].apply(remove_char)
    mlb_city_df = mlb_city_df.dropna()
    mlb_city_df

    def mlb_city(x):
        x = x.strip()
        if 'City' in x:
            return x.split('City')[0].strip()
        elif '–' in x:
            return x.split('–')[0].strip()
        elif 'Bay' in x:
            return x.split('Bay')[0].strip()
        elif ',' in x:
            return x.split(',')[0].strip()
        return x


    mlb_city_df['Team City'] = mlb_city_df['Metropolitan area'].apply(mlb_city)
    mlb_city_df

    mlb_df_2018['Team City'] = ''
    for l1,c1 in mlb_city_df['Team City'].items():
        for l2,c2 in mlb_df_2018['team'].items():
            if c1 in c2:
                mlb_df_2018['Team City'][l2] = c1
            elif 'Minnesota Twins' in c2:
                mlb_df_2018['Team City'][l2] = 'Minneapolis'
            elif 'Oakland Athletics' in c2:
                mlb_df_2018['Team City'][l2] = 'San Francisco'
            elif 'Texas Rangers' in c2:
                mlb_df_2018['Team City'][l2] = 'Dallas'
            elif 'Colorado Rockies' in c2:
                mlb_df_2018['Team City'][l2] = 'Denver'
            elif 'Arizona Diamondbacks' in c2:
                mlb_df_2018['Team City'][l2] = 'Phoenix'    

    mlb_df_2018
    mlb_df_2018['W'] = [int(x) for x in mlb_df_2018['W']]
    mlb_df_2018['L'] = [int(x) for x in mlb_df_2018['L']]

    mlb_df = pd.merge(mlb_city_df, mlb_df_2018, how='inner', on='Team City')
    mlb_df
    mlb_df['Ratio'] = (mlb_df['W'])/((mlb_df['W'])+(mlb_df['L']))

    mlb_result_df = mlb_df.groupby(['Metropolitan area', 'Population']).agg({'Ratio':np.mean})
    mlb_result_df = mlb_result_df.reset_index()

    
    population_by_region = mlb_result_df['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = mlb_result_df['Ratio']  # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    raise NotImplementedError()
#mlb_correlation()

0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [19]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re



           



def nfl_correlation():
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]


    nfl_df_2018 = nfl_df[nfl_df['year'] == 2018]
    #mlb_df_2018

    nfl_city_df = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NFL']]
    nfl_city_df =  nfl_city_df.rename(columns={'Population (2016 est.)[8]':'Population'})
    nfl_city_df['Population'] = [int(x) for x in nfl_city_df['Population']]

    def remove_char(x):
        x = str(x).strip()
        if '[note' in x:
            lst = x.split('[')
            if lst[0]=='':
                return np.nan
            return lst[0].strip()
        if '—' in x:
            x = np.nan
        elif 'note 22' in x:
            x = np.nan
        return x

    nfl_city_df['NFL'] = nfl_city_df['NFL'].apply(remove_char)
    nfl_city_df = nfl_city_df.dropna()
    nfl_city_df
    def nfl_city(x):
        x = x.strip()
        if 'City' in x:
            return x.split('City')[0].strip()
        elif '–' in x:
            return x.split('–')[0].strip()
        elif 'Bay' in x:
            return x.split('Bay')[0].strip()
        elif ',' in x:
            return x.split(',')[0].strip()
        return x


    nfl_city_df['Team City'] = nfl_city_df['Metropolitan area'].apply(nfl_city)
    nfl_city_df
#Dropping division rows
    nfl_df_2018 = nfl_df_2018.drop([0,5,10,15,20,25,30,35], axis=0)
    nfl_df_2018['Team City'] = ''
    for l1,c1 in nfl_city_df['Team City'].items():
        for l2,c2 in nfl_df_2018['team'].items():
            if c1 in c2:
                nfl_df_2018['Team City'][l2] = c1

            elif 'New England Patriots' in c2:
                nfl_df_2018['Team City'][l2] = 'Boston'
            elif 'Tennessee Titans' in c2:
                nfl_df_2018['Team City'][l2] = 'Nashville'
            elif 'Oakland Raiders' in c2:
                nfl_df_2018['Team City'][l2] = 'San Francisco'
            elif 'Minnesota Vikings' in c2:
                nfl_df_2018['Team City'][l2] = 'Minneapolis'
            elif 'Carolina Panthers' in c2:
                nfl_df_2018['Team City'][l2] = 'Charlotte'
            elif 'Arizona Cardinals' in c2:
                nfl_df_2018['Team City'][l2] = 'Phoenix'   

    #nfl_df_2018
    #['New England Patriots','Tennessee Titans','Oakland Raiders	','Minnesota Vikings','Carolina Panthers','Arizona Cardinals']
    nfl_df_2018['W'] = [int(x) for x in nfl_df_2018['W']]
    nfl_df_2018['L'] = [int(x) for x in nfl_df_2018['L']]
    # YOUR CODE HERE
    
    nfl_df = pd.merge(nfl_city_df, nfl_df_2018, how='inner', on='Team City')
    
    nfl_df['Ratio'] = (nfl_df['W'])/((nfl_df['W'])+(nfl_df['L']))

    nfl_result_df = nfl_df.groupby(['Metropolitan area', 'Population']).agg({'Ratio':np.mean})
    nfl_result_df = nfl_result_df.reset_index()
    
    
    population_by_region = nfl_result_df['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = nfl_result_df['Ratio'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    raise NotImplementedError()
#nfl_correlation()

0.004922112149349393

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values